# Vast.ai
Create account, add card & add money (for security reasons, you could remove the card after adding money).

Generate ssh key on local machine & follow the instructions. Store the public key to vastai
```bash
ssh-keygen -t ed25519
```

Upload this notebook to a machine that has access to a jupyter notebook server.


## Using Colab

see instructions from [Huggingface](https://colab.research.google.com/github/huggingface/notebooks/blob/main/lerobot/training-smolvla.ipynb#scrollTo=QlKjL1X5t_zM)

## Install LeRobot

In [ ]:
!git clone https://github.com/JPRijstenbil/lerobot
!cd lerobot && pip install -e .

### RTX50XX

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

### RTX 40XX

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

In [ ]:
# Optionally: check if pytorch works correctly
import torch
import numpy as np
torch.cuda.is_available() # Should return 'True'

device = torch.device('cuda')
a = np.array([[1,2],[2,3]])
b = torch.from_numpy(a)
b.to(device) # Should return a cuda tensor

## Logins

In [ ]:
WB_KEY = "<W&B API Key>" # Replace with your Weights & Biases API key
HF_KEY = "<Hugging Face API Key>" # Replace with your Hugging Face API key

In [ ]:
import wandb
from huggingface_hub import whoami

wandb.login(key=WB_KEY)

info = whoami()
HF_USER = info['name']
print(f"✅ You are logged in as {HF_USER}")

!huggingface-cli login --token $HF_KEY

## Training ACT

Speed = 100k steps in 1.5 hour (A100 GPU)

In [ ]:
!cd lerobot
!python lerobot/scripts/train.py \
  --dataset.repo_id=${HF_USER}/il_gym0 \
  --policy.type=act \
  --output_dir=outputs/train/il_sim_test0 \
  --job_name=il_sim_test \
  --policy.device=cuda \
  --wandb.enable=true

In [ ]:
!huggingface-cli upload ${HF_USER}/il_sim_test0 /content/lerobot/outputs/train/il_sim_test0/checkpoints/last/pretrained_model

## Finetuning SmolVLA

Speed = 20k steps in 5 hours (A100 GPU)

In [ ]:
# install SmolVLA dependencies
!cd lerobot && pip install -e ".[smolvla]"

!cd lerobot
!python lerobot/scripts/train.py \
  --policy.path=lerobot/smolvla_base \
  --dataset.repo_id=${HF_USER}/mydataset \
  --batch_size=64 \
  --steps=20000 \
  --output_dir=outputs/train/my_smolvla \
  --job_name=my_smolvla_training \
  --policy.device=cuda \
  --wandb.enable=true

In [ ]:
# Remember to set this to the correct folder

!huggingface-cli upload ${HF_USER}/my_smolvla /content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model

## Training Groot N1.5